In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir(os.getcwd())

In [2]:
os.getcwd()

'/mnt/github/RiboVsPolyA/examples'

In [3]:
#Author: Ioannis Anastopoulos
from sklearn.metrics import r2_score,confusion_matrix, accuracy_score,roc_curve,auc,precision_recall_fscore_support,f1_score, average_precision_score
from scipy.stats import pearsonr
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def F_score(Y_true,predict_classes ,classes=None, plot=False,average=None):
    if classes is None:
        classes=range(len(set(Y_true)))
    eval_metrics_matrix=precision_recall_fscore_support(Y_true,predict_classes,labels=classes,average=average)
    f_score_df=pd.DataFrame(eval_metrics_matrix[2], columns=['F_score'],index=classes)
    f_score_df=f_score_df.drop_duplicates()

    if plot:
        plt.figure(figsize=(10,10))
        ax=f_score_df.plot(kind='bar', figsize=(7,10))
        for p in ax.patches:
            ax.annotate(str(round(p.get_height(),2)), (p.get_x() * 1.005, p.get_height() * 1.005))
        plt.title('F1_score')
        plt.show()
        plt.close()
    return f_score_df

def confusion(Y_true,predict_classes,classes=None,plot=False, title='Confusion Matrix',fontsize=16,cmap='tab20',normalize=False):

    if classes is None:
        classes=range(len(set(predict_classes)))

    conf_matrix=confusion_matrix(Y_true, predict_classes, labels=classes)

    if normalize:
        total=conf_matrix.sum(axis=0) #summing actual predictions
        conf_matrix= conf_matrix/total

    df_cm = pd.DataFrame(conf_matrix, index=classes,
                  columns=classes)
    #sns.set(font_scale=5)#for label size
    if plot:
        plt.figure(figsize=(10,10))
        sns.heatmap(df_cm, annot=True,annot_kws={"size":fontsize*(1/2) },cmap=cmap)# font size
        #fig=conf_hm.get_figure()
        plt.yticks(fontsize=fontsize+4)
        plt.xticks(fontsize=fontsize+4)
        plt.title(title, fontsize=fontsize+10)
        #fig.savefig(out,dpi=300)
        plt.show()
        plt.close()

    return conf_matrix

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

def ROC(y_true, y_pred,classes=None,title=None,plot=False):

    plt.figure(figsize=(10,10))
    lw=4

    if isinstance(y_true, pd.DataFrame):
        y_true=y_true.values
    if isinstance(y_pred, pd.DataFrame):
        y_pred=y_pred.values


    if classes is None:
        classes=set(list(y_true.flatten())) #removing dups, and i think itll only work for binary cases

    if len(classes)>2:
        #y_true = to_categorical(y_true, len(classes))
        #y_pred = to_categorical(y_pred, len(classes))
        
        fpr = dict()
        tpr = dict()
        roc_auc = dict()

        for i,cl in enumerate(classes):
            fpr[cl], tpr[cl], _ = roc_curve(y_true[:,i], y_pred[:, i])
            roc_auc[cl] = auc(fpr[cl], tpr[cl])
        
        if plot: 
            colors = plt.cm.get_cmap('tab20',len(classes)) #best line of code IN THE UNIVERSE

            for i, color in zip(range(len(classes)), colors.colors):
                cl = classes[i]
                plt.plot(fpr[cl], tpr[cl], color=colors.colors[i], lw=lw,
                     label='ROC curve of class {0} (area = {1:0.2f})'
                     ''.format(cl, roc_auc[cl]))
            plt.plot([0, 1], [0, 1], 'k--', lw=lw)
            plt.xlim([-0.03, 1.05])
            plt.ylim([-0.03, 1.05])
            plt.xlabel('False Positive Rate', fontsize=10)
            plt.ylabel('True Positive Rate',fontsize=10)
            plt.title(title, fontsize=12)
            plt.xticks(fontsize=10)
            plt.yticks(fontsize=10)
            plt.legend(loc="lower right",prop={'size': 15})
            #plt.savefig('/projects/sysbio/users/ianastop/cm_region_results/plots/%s_ROC.png'%self.out,dpi=300)
            plt.show()
            plt.close()

        return (fpr, tpr, roc_auc)

    else:
        fpr, tpr, _ = roc_curve(y_true, y_pred)
        roc_auc = auc(fpr, tpr)
        if plot:
            plt.plot(fpr, tpr, color='darkorange', lw=4, label='ROC curve (area = %0.2f)' % roc_auc)

            plt.plot([0, 1], [0, 1], 'k--', lw=lw)
            plt.xlim([-0.03, 1.05])
            plt.ylim([-0.03, 1.05])
            plt.xlabel('False Positive Rate', fontsize=10)
            plt.ylabel('True Positive Rate',fontsize=10)
            plt.title(title, fontsize=12)
            plt.xticks(fontsize=10)
            plt.yticks(fontsize=10)
            plt.legend(loc="lower right",prop={'size': 10})
            #plt.savefig('/projects/sysbio/users/ianastop/cm_region_results/plots/%s_ROC.png'%self.out,dpi=300)
            return( fpr, tpr, roc_auc)
            plt.show()
            plt.close()
        print(fpr, tpr)
        return fpr,tpr,roc_auc


def regression_eval(y_true,y_pred, axis=None):
    '''
    Function computes pearson, r2 and rmse row wise or column wise
    Returns a list for each metric for each row or column

    axis =0/1 if the vector predicted has multiple rows and columns: for expression matrices
    if the vectors has multiple samples predicted for 1 variable : for drug response vector of each drug individually
    '''
    #calculating metrics on a per sample basis
    if axis ==0:
        r_list = []
        r2_list = []
        rmse_list = []
        for i, vec in enumerate(y_true):
            r_list +=[pearsonr(vec, y_pred[i])[0]]
            r2_list += [r2_score(vec, y_pred[i])]
            rmse_list += [rmse(vec,y_pred[i])]
        r = np.mean(r_list)
        r2 = np.mean(r2_list)
        rmse_score = np.mean(rmse_list)
    elif axis == 1:
        r_list = []
        r2_list = []
        rmse_list = []
        for i, vec in enumerate(y_true.T):
            r_list +=[pearsonr(vec, y_pred.T[i])[0]]
            r2_list += [r2_score(vec, y_pred.T[i])]
            rmse_list += [rmse(vec,y_pred.T[i])]
        r = np.mean(r_list)
        r2 = np.mean(r2_list)
        rmse_score = np.mean(rmse_list)
    elif axis is None:
        r, _ = pearsonr(y_true.flatten(), y_pred.flatten())
        r2 = r2_score(y_true.flatten(), y_pred.flatten())
        rmse_score = rmse(y_true.flatten(), y_pred.flatten())
    return r,r2,rmse_score

def classification_eval(y_true,y_pred, y_proba,classes=None):
    if len(y_proba.shape)>=2:
        y_pred = np.argmax(y_proba, axis=1)
    acc = accuracy_score(y_true,y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    roc_auc = ROC(y_true, y_proba,classes=classes)[-1]
    precision = average_precision_score(y_true, y_proba, average='weighted')
    
    return acc, f1, roc_auc, precision
        
def node_classification_eval():
    pass 

def corr_plot(y_true, y_pred, title='Fig Title', ylabel='Y label', xlabel='X label', out_fn='out.png', log_values=False):
    fig, axes = plt.subplots(nrows=3, ncols=3,figsize=(10,10))
    i=0
    j=0
    np.random.seed(42)
    for idx,sample_idx in enumerate(np.random.choice(np.arange(y_true.shape[0]),9)):

        ax_row =axes[i] #select row  to plot in
        ax = ax_row[j]
        if log_values:
            observed = [np.log(x+1) for x in y_true[int(sample_idx/2)]]
            predicted = [np.log(x+1) for x in y_pred[int(sample_idx/2)]]
        else:
            observed = y_true[int(sample_idx/2)]
            predicted = y_pred[int(sample_idx/2)]
            
        ax.scatter(observed, predicted, s=1, alpha=0.6, marker='o')

        ax.set_title("Random sample %d"%(idx+1))

        if j==2:
            i+=1
            j=0
        else:
            j+=1
        r = pearsonr(y_true[int(sample_idx/2)], y_pred[int(sample_idx/2)])[0]
        r2 = r2_score(y_true[int(sample_idx/2)], y_pred[int(sample_idx/2)])
        ax.text(min(observed), 0.95*max(predicted), 'pearson coeff: %.3f'%r)
        ax.text(min(observed), 0.8*max(predicted), r'$R^2$: %.3f'%r2)

    #     handles, labels = ax.get_legend_handles_labels()
    #     by_label = dict(zip(labels, handles))
    #     ax.legend(by_label.values(), by_label.keys())
        ax.set_ylabel(ylabel, fontsize=10)
        ax.set_xlabel(xlabel, fontsize=10)

    st = fig.suptitle(title, fontsize=15)
    st.set_y(1.05)
    fig.subplots_adjust(top=0.85)

    plt.tight_layout()
    plt.savefig(out_fn, dpi=300)
    plt.show()
    plt.close()


/home/ubuntu/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Using Treehouse Pipeline

In [4]:
srp_tpm_df = pd.read_csv("../results/SRP026013_TPM_log2_plus_1.TH.balanced_maxdepth1_results.tsv", index_col=0, sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '../results/SRP026013_TPM_log2_plus_1.TH.balanced_maxdepth1_results.tsv'

In [ ]:
srp_tpm_df.shape

In [ ]:
true_labels = np.array([0]*srp_tpm_df.shape[0])
pred_labels = srp_tpm_df.Ribo.values
pred_proba = srp_tpm_df.Proba_1.values

In [ ]:
true_labels

In [ ]:
pred_labels

In [ ]:
srp_tpm_df.iloc[11]

In [ ]:
pred_labels==0

In [ ]:
pred_proba

In [ ]:
acc, f1, roc_auc, precision= classification_eval(true_labels,pred_labels,pred_proba, classes=[0,1])
print('Accuracy: {}\nF1-score: {}'.format(acc, f1))

## Using refine.bio Pipeline

In [9]:
srp_tpm_df = pd.read_csv("../results/SRP026013_TPM_log2_plus_1.balanced_maxdepth1_results.tsv", index_col=0, sep='\t')

In [10]:
srp_tpm_df.shape

(16, 3)

In [11]:
true_labels = np.array([0]*srp_tpm_df.shape[0])
pred_labels = srp_tpm_df.Ribo.values
pred_proba = srp_tpm_df.Proba_1.values

In [12]:
acc, f1, roc_auc, precision= classification_eval(true_labels,pred_labels,pred_proba, classes=[0,1])
print('Accuracy: {}\nF1-score: {}'.format(acc, f1))

[0.     0.0625 0.8125 0.9375 1.    ] [nan nan nan nan nan]
Accuracy: 0.0625
F1-score: 0.11764705882352941


<Figure size 720x720 with 0 Axes>